In [38]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing


In [39]:
filepath = r"D:\FPL-Lineup-Prediction\data\converted_data\gameweek_data.csv"

df = pd.read_csv(filepath,index_col=0)


In [40]:
df.head()

,player_id,player_name,now_cost,position,team,gw1,gw2,gw3,gw4,gw5,gw6,gw7,gw8,gw9,gw10,gw11,gw12
0,1,Raya,59,GKP,Arsenal,10.0,6.0,2.0,6.0,2.0,2.0,6.0,6.0,6.0,6.0,1.0,2
1,2,Arrizabalaga,42,GKP,Arsenal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,3,Hein,40,GKP,Arsenal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,4,Setford,40,GKP,Arsenal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,5,Gabriel,64,DEF,Arsenal,6.0,6.0,2.0,9.0,2.0,13.0,9.0,12.0,9.0,12.0,1.0,0


In [41]:
df['now_cost'] = df['now_cost'] / 10

In [42]:
gw_cols = [f"gw{i}" for i in range(1, 13)]

df[gw_cols] = df[gw_cols].fillna(0)

In [43]:
gw_cols = [f"gw{i}" for i in range(1,13)]
df["avg_pt"] = df[gw_cols].mean(axis = 1).round(2)

In [44]:
df.loc[df['avg_pt'].idxmax()]

player_id           430
player_name     Haaland
now_cost           14.9
position            FWD
team           Man City
gw1                13.0
gw2                 2.0
gw3                 9.0
gw4                13.0
gw5                 9.0
gw6                16.0
gw7                 8.0
gw8                13.0
gw9                 2.0
gw10               13.0
gw11                4.0
gw12                  2
avg_pt             8.67
Name: 429, dtype: object

In [45]:
n = len(gw_cols)
weights = np.arange(1,n+1)

weights = weights / weights.sum()

df['weighted_avg'] = df[gw_cols].values.dot(weights).round(2)

In [46]:
df.head()

,player_id,player_name,now_cost,position,team,gw1,gw2,gw3,gw4,gw5,gw6,gw7,gw8,gw9,gw10,gw11,gw12,avg_pt,weighted_avg
0,1,Raya,5.9,GKP,Arsenal,10.0,6.0,2.0,6.0,2.0,2.0,6.0,6.0,6.0,6.0,1.0,2,4.58,4.01
1,2,Arrizabalaga,4.2,GKP,Arsenal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.00,0.00
2,3,Hein,4.0,GKP,Arsenal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.00,0.00
3,4,Setford,4.0,GKP,Arsenal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.00,0.00
4,5,Gabriel,6.4,DEF,Arsenal,6.0,6.0,2.0,9.0,2.0,13.0,9.0,12.0,9.0,12.0,1.0,0,6.75,6.65


In [47]:
top_10 = df.nlargest(10, 'weighted_avg')
print(top_10)

     player_id player_name  now_cost position            team   gw1  gw2  gw3  \
429        430     Haaland      14.9      FWD        Man City  13.0  2.0  9.0   
255        256       Muñoz       5.9      DEF  Crystal Palace   5.0  5.0  9.0   
4            5     Gabriel       6.4      DEF         Arsenal   6.0  6.0  2.0   
265        266         Eze       7.7      MID         Arsenal   3.0  0.0  1.0   
20          21        Rice       7.0      MID         Arsenal   6.0  6.0  4.0   
294        295       Keane       4.5      DEF         Everton   2.0  8.0  3.0   
19          20    Trossard       6.9      MID         Arsenal   0.0  1.0  0.0   
235        236        Neto       7.1      MID         Chelsea   3.0  7.0  3.0   
135        136      Thiago       6.6      FWD       Brentford   6.0  2.0  6.0   
487        488    Bruno G.       6.6      MID       Newcastle   3.0  6.0  3.0   

      gw4  gw5   gw6   gw7   gw8   gw9  gw10  gw11  gw12  avg_pt  weighted_avg  
429  13.0  9.0  16.0   8.0 

In [48]:
exp_smoothing_values = []

for _, row in df.iterrows():
    ts = row[gw_cols].values.astype(float)

    # Fit Holt's additive trend model
    model = ExponentialSmoothing(ts, trend="add", seasonal=None)

    try:
        fit = model.fit()
        horizon = 38 - len(ts)
        forecast = fit.forecast(horizon)
        exp_val = forecast.mean()
    except:
        exp_val = np.nan   # fallback if model fails

    exp_smoothing_values.append(exp_val)

df["exp_smoothing"] = exp_smoothing_values

d:\FPL-Lineup-Prediction\venv\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1380: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + k * 2
d:\FPL-Lineup-Prediction\venv\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1387: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)
d:\FPL-Lineup-Prediction\venv\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1380: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + k * 2
d:\FPL-Lineup-Prediction\venv\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1387: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)
d:\FPL-Lineup-Prediction\venv\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1380: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + k * 2
d:\

In [49]:
df.head()

,player_id,player_name,now_cost,position,team,gw1,gw2,gw3,gw4,gw5,gw6,gw7,gw8,gw9,gw10,gw11,gw12,avg_pt,weighted_avg,exp_smoothing
0,1,Raya,5.9,GKP,Arsenal,10.0,6.0,2.0,6.0,2.0,2.0,6.0,6.0,6.0,6.0,1.0,2,4.58,4.01,-1.329255
1,2,Arrizabalaga,4.2,GKP,Arsenal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.000000
2,3,Hein,4.0,GKP,Arsenal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.000000
3,4,Setford,4.0,GKP,Arsenal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.000000
4,5,Gabriel,6.4,DEF,Arsenal,6.0,6.0,2.0,9.0,2.0,13.0,9.0,12.0,9.0,12.0,1.0,0,6.75,6.65,5.753497


In [50]:
top_10 = df.nlargest(10, 'exp_smoothing')
print(top_10)

     player_id    player_name  now_cost position           team  gw1  gw2  \
486        487         Barnes       6.3      MID      Newcastle  3.0  2.0   
670        671         Wilson       5.8      FWD       West Ham  1.0  1.0   
505        506        Murillo       5.3      DEF  Nott'm Forest  2.0  1.0   
504        505     Milenković       5.2      DEF  Nott'm Forest  2.0  2.0   
364        365         Nmecha       5.0      FWD          Leeds  5.0  1.0   
19          20       Trossard       6.9      MID        Arsenal  0.0  1.0   
265        266            Eze       7.7      MID        Arsenal  3.0  0.0   
241        242  Dewsbury-Hall       4.9      MID        Everton  2.0  2.0   
291        292      Mykolenko       4.9      DEF        Everton  0.0  0.0   
238        239        Gittens       6.1      MID        Chelsea  1.0  1.0   

      gw3  gw4  gw5  gw6  gw7   gw8   gw9  gw10  gw11  gw12  avg_pt  \
486   1.0  3.0  1.0  1.0  1.0   1.0   1.0   1.0   9.0    15    3.25   
670   7.0 

In [51]:
df.columns

Index(['player_id', 'player_name', 'now_cost', 'position', 'team', 'gw1',
       'gw2', 'gw3', 'gw4', 'gw5', 'gw6', 'gw7', 'gw8', 'gw9', 'gw10', 'gw11',
       'gw12', 'avg_pt', 'weighted_avg', 'exp_smoothing'],
      dtype='object')

In [52]:
df.isna().sum()

player_id        0
player_name      0
now_cost         0
position         0
team             0
gw1              0
gw2              0
gw3              0
gw4              0
gw5              0
gw6              0
gw7              0
gw8              0
gw9              0
gw10             0
gw11             0
gw12             0
avg_pt           0
weighted_avg     0
exp_smoothing    0
dtype: int64

In [53]:
df['position'].unique()

array(['GKP', 'DEF', 'MID', 'FWD'], dtype=object)

In [54]:
import pulp
import pandas as pd

def select_xi_with_captain(
    df: pd.DataFrame,
    score_col: str,
    budget: float = 83.5,          # 83.5m in FPL integer scale
    min_score: float = 0.05       # filter garbage players
):
    """
    Solves the FPL starting XI + captain selection using Integer Programming.
    """

    # -----------------------------
    # 1. Sanity checks
    # -----------------------------
    required_cols = {"player_id", "player_name", "team", "position", "now_cost", score_col}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"Missing columns: {missing}")


    # Filter useless players (CRITICAL)
    df = df[df[score_col] > min_score].copy()
    if df.empty:
        raise ValueError("All players filtered out — check score_col values")

    # Normalize position labels (GK vs GKP)
    df["position"] = df["position"].replace({"GKP": "GK"})

    # -----------------------------
    # 2. Model
    # -----------------------------
    model = pulp.LpProblem("FPL_XI_with_Captain", pulp.LpMaximize)

    x = {i: pulp.LpVariable(f"x_{i}", cat="Binary") for i in df.index}
    y = {i: pulp.LpVariable(f"y_{i}", cat="Binary") for i in df.index}

    # Objective (captain doubles score)
    model += pulp.lpSum(
        df.loc[i, score_col] * (x[i] + y[i])
        for i in df.index
    )

    # -----------------------------
    # 3. Constraints
    # -----------------------------

    # Exactly 11 players
    model += pulp.lpSum(x[i] for i in df.index) == 11

    # Budget
    model += pulp.lpSum(df.loc[i, "now_cost"] * x[i] for i in df.index) <= budget

    # Exactly one captain
    model += pulp.lpSum(y[i] for i in df.index) == 1

    # Captain must be selected
    for i in df.index:
        model += y[i] <= x[i]

    # Formation constraints
    pos_limits = {
        "GK":  (1, 1),
        "DEF": (3, 5),
        "MID": (3, 5),
        "FWD": (1, 3)
    }

    for pos, (mn, mx) in pos_limits.items():
        idx = df.index[df["position"] == pos]
        if len(idx) == 0:
            raise ValueError(f"No players found for position {pos}")
        model += pulp.lpSum(x[i] for i in idx) >= mn
        model += pulp.lpSum(x[i] for i in idx) <= mx

    # Max 3 players per team
    for team in df["team"].unique():
        idx = df.index[df["team"] == team]
        model += pulp.lpSum(x[i] for i in idx) <= 3

    # -----------------------------
    # 4. Solve
    # -----------------------------
    model.solve(pulp.PULP_CBC_CMD(msg=False))

    status = pulp.LpStatus[model.status]
    if status != "Optimal":
        raise RuntimeError(f"Solver failed: {status}")

    # -----------------------------
    # 5. Extract solution
    # -----------------------------
    selected_idx = [i for i in df.index if x[i].value() == 1]
    selected = df.loc[selected_idx].copy()

    selected["is_captain"] = [int(y[i].value()) for i in selected_idx]

    # Final hard assertions (DO NOT REMOVE)
    assert selected.shape[0] == 11
    assert selected["is_captain"].sum() == 1
    assert selected["now_cost"].sum() <= budget

    return selected.sort_values(by=score_col, ascending=False)


In [55]:
xi = select_xi_with_captain(
    df,
    score_col="weighted_avg",
    budget=83.5      # NOT 83.5
)

print(xi[[
    "player_name",
    "position",
    "team",
    "now_cost",
    "weighted_avg",
    "is_captain"
]])

print("Total cost:", xi["now_cost"].sum())
print("Captain:", xi.loc[xi["is_captain"] == 1, "player_name"].iloc[0])


    player_name position            team  now_cost  weighted_avg  is_captain
429     Haaland      FWD        Man City      14.9          7.86           1
255       Muñoz      DEF  Crystal Palace       5.9          6.82           0
4       Gabriel      DEF         Arsenal       6.4          6.65           0
265         Eze      MID         Arsenal       7.7          6.65           0
20         Rice      MID         Arsenal       7.0          6.29           0
294       Keane      DEF         Everton       4.5          6.15           0
235        Neto      MID         Chelsea       7.1          5.87           0
135      Thiago      FWD       Brentford       6.6          5.82           0
487    Bruno G.      MID       Newcastle       6.6          5.79           0
256     Lacroix      DEF  Crystal Palace       5.0          5.74           0
31     Martinez       GK     Aston Villa       5.0          4.56           0
Total cost: 76.7
Captain: Haaland


In [56]:
xi = select_xi_with_captain(
    df,
    score_col="avg_pt",
    budget=83.5      # NOT 83.5
)

print(xi[[
    "player_name",
    "position",
    "team",
    "now_cost",
    "weighted_avg",
    "is_captain"
]])

print("Total cost:", xi["now_cost"].sum())
print("Captain:", xi.loc[xi["is_captain"] == 1, "player_name"].iloc[0])


    player_name position            team  now_cost  weighted_avg  is_captain
429     Haaland      FWD        Man City      14.9          7.86           1
4       Gabriel      DEF         Arsenal       6.4          6.65           0
81      Semenyo      MID     Bournemouth       7.9          4.58           0
255       Muñoz      DEF  Crystal Palace       5.9          6.82           0
7      J.Timber      DEF         Arsenal       6.3          5.29           0
20         Rice      MID         Arsenal       7.0          6.29           0
259       Guéhi      DEF  Crystal Palace       5.1          5.19           0
256     Lacroix      DEF  Crystal Palace       5.0          5.74           0
135      Thiago      FWD       Brentford       6.6          5.82           0
159      Minteh      MID        Brighton       6.2          5.74           0
669       Roefs       GK      Sunderland       4.7          4.21           0
Total cost: 76.0
Captain: Haaland


In [57]:
xi = select_xi_with_captain(
    df,
    score_col="exp_smoothing",
    budget=83.5      # NOT 83.5
)

print(xi[[
    "player_name",
    "position",
    "team",
    "now_cost",
    "weighted_avg",
    "is_captain"
]])

print("Total cost:", xi["now_cost"].sum())
print("Captain:", xi.loc[xi["is_captain"] == 1, "player_name"].iloc[0])


       player_name position           team  now_cost  weighted_avg  is_captain
486         Barnes      MID      Newcastle       6.3          4.44           1
670         Wilson      FWD       West Ham       5.8          3.74           0
505        Murillo      DEF  Nott'm Forest       5.3          3.60           0
504     Milenković      DEF  Nott'm Forest       5.2          2.78           0
364         Nmecha      FWD          Leeds       5.0          3.15           0
19        Trossard      MID        Arsenal       6.9          6.04           0
265            Eze      MID        Arsenal       7.7          6.65           0
241  Dewsbury-Hall      MID        Everton       4.9          4.74           0
291      Mykolenko      DEF        Everton       4.9          3.24           0
238        Gittens      MID        Chelsea       6.1          1.63           0
31        Martinez       GK    Aston Villa       5.0          4.56           0
Total cost: 63.1
Captain: Barnes
